## Weaviate Query Agent with Pydantic

This notebook will show you how to define the Weaviate Query Agent as a tool through Pydantic.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
1. Install Pydantic with `pip install pydantic`. We're using version `2.10.6` at the time of writing this notebook.
1. Install the Weaviate Agents package with `pip install weaviate-agents`
1. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [ ]:
import weaviate
from weaviate_agents.query import QueryAgent
import os

from pydantic_ai import Agent, RunContext
from typing import Optional, Dict, Any
import asyncio
from pydantic import BaseModel

In [1]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

### Define Query Agent tool

In [37]:
# Define credential structure
class WeaviateCredentials(BaseModel):
    weaviate_url: str
    weaviate_api_key: str 
    openai_api_key: str
    collections: list[str]

class HelpfulResponse(BaseModel):
    helpful_response: str

# Initialize the Agent with GPT-4 and typed credentials
agent = Agent(
    'openai:gpt-4o',
    deps_type=WeaviateCredentials,
    result_type=HelpfulResponse,
    system_prompt=(
        "You are a helpful assistant that can search a knowledge base to answer questions. "
        "Use the search tool to find relevant information, then provide a comprehensive answer. "
        "If the search doesn't return useful information, acknowledge the limitations."
    ),
)

@agent.tool
def send_query_agent_request(ctx: RunContext[WeaviateCredentials], query: str) -> str:
    """
    Send a query to the Weaviate database and get the response.
    Using tool_plain which works better with synchronous functions.

    Args:
        ctx: The run context containing Weaviate credentials and configuration.
        query (str): The question or query to search for in the database.

    Returns:
        str: The response from the database containing relevant information.
    """
    try:
        # Get credentials from dependencies, with environment variables as fallback
        creds = ctx.deps
        weaviate_url = creds.weaviate_url or os.getenv("WEAVIATE_URL")
        weaviate_api_key = creds.weaviate_api_key or os.getenv("WEAVIATE_API_KEY")
        openai_api_key = creds.openai_api_key or os.getenv("OPENAI_API_KEY")
        collections = creds.collections or ["Blogs"]
        
        if not all([weaviate_url, weaviate_api_key, openai_api_key]):
            raise ValueError("Missing required credentials")
            
        # Connect to Weaviate Cloud instance
        weaviate_client = weaviate.connect_to_weaviate_cloud(
            cluster_url=weaviate_url,
            auth_credentials=weaviate.auth.AuthApiKey(weaviate_api_key),
            headers={"X-OpenAI-Api-Key": openai_api_key},
        )
        
        # connect the query agent to your Weaviate collection(s)
        query_agent = QueryAgent(
            client=weaviate_client,
            collections=collections
        )
        
        result = query_agent.run(query)
        return result.final_answer
    except Exception as e:
        return f"Error querying Weaviate: {str(e)}"

### Query time

In [41]:
custom_credentials = WeaviateCredentials(
    weaviate_url=os.getenv("WEAVIATE_URL"),
    weaviate_api_key=os.getenv("WEAVIATE_API_KEY"), 
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    collections=["Blogs"]
)

# Use await directly in your notebook cell
response = await agent.run(
    "How do I run Weaviate with Docker?",
    deps=custom_credentials
)
print(response.data.helpful_response)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/selector_events.py:710: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=89 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/selector_events.py:710: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=90 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


To run Weaviate with Docker, follow these steps:

1. **Ensure Docker is Installed:** Make sure Docker is installed on your system. You can download it from the [Docker website](https://www.docker.com/).

2. **Fetch the Weaviate Docker Image:** Open your terminal and pull the Weaviate image from Docker Hub by executing the following command:
   ```bash
   docker pull semitechnologies/weaviate
   ```

3. **Run the Weaviate Container:** Start the Weaviate container using the command:
   ```bash
   docker run -d --name weaviate -p 8080:8080 semitechnologies/weaviate
   ```
   - This command runs the container in detached mode (`-d`).
   - It names the container `weaviate` for easy reference.
   - It maps port 8080 on your local machine to port 8080 on the container, allowing you to access the Weaviate instance at `http://localhost:8080`.

4. **Verify the Weaviate Service:** You can check if Weaviate is running by visiting `http://localhost:8080/` in your web browser, or by using curl with:

In [42]:
response.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="You are a helpful assistant that can search a knowledge base to answer questions. Use the search tool to find relevant information, then provide a comprehensive answer. If the search doesn't return useful information, acknowledge the limitations.", part_kind='system-prompt'), UserPromptPart(content='How do I run Weaviate with Docker?', timestamp=datetime.datetime(2025, 3, 4, 13, 2, 6, 8130, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='send_query_agent_request', args=ArgsJson(args_json='{"query":"How to run Weaviate with Docker"}'), tool_call_id='call_4JrAGH6Ptna8dpfyyCeHIqk5', part_kind='tool-call')], timestamp=datetime.datetime(2025, 3, 4, 13, 2, 6, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='send_query_agent_request', content="To run Weaviate with Docker, you need Docker installed on your system. Follow these st